### Sample code of predictive analytics ML pipeline in AI digital marketing project for company

### Get API keys from GitHub
Complete this step for "cloud automation"

In [ ]:
import requests
import io
import json

# Username of team GitHub account
username = '<Team GitHub account username>'

# Personal Access Token (PAO) from team GitHub account
token = '<Team GitHub account PAO>'

github_session = requests.Session()
github_session.auth = (username, token)

# Key1: Google Bigquery query key
url1 = '<URL to request API key1>' 
key1 = github_session.get(url1).json()

with open("key1.json", "w") as key1_file:  # Save as json file
    json.dump(key1, key1_file)

# Key2: GA management API key
url2 = '<URL to request API key2>'
key2 = github_session.get(url2).json()

with open("key2.json", "w") as key2_file: 
    json.dump(key2, key2_file)

In [39]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import date
from datetime import timedelta
from openpyxl import Workbook
import warnings
warnings.filterwarnings('ignore')
np.set_printoptions(suppress=True)
pd.set_option('display.max_columns', None)

In [41]:
from datetime import date
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif, chi2
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
# from xgboost import XGBClassifier
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import StackingClassifier
import matplotlib.pyplot as plt
# import seaborn as sns

### Connect to company data stored in Google BigQuery 

In [43]:
# Connect Google BigQuery with Python
import os
from google.cloud import bigquery as bq

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'key1.json'  # Query key from GitHub (json file)

# client = bq.Client()

### Update realtime website behavior data in BigQuery

In [ ]:
class Update_data:

    def __init__(self, table):
        self.table = table  # company data source on BigQuery
        self.client = bq.Client()


    # Update raw data
    def update_raw(self):
        
        update_raw_query = self.client.query(
        f'''
        CREATE OR REPLACE TABLE 
        `<company data table on BigQuery: self.table>` AS

        SELECT *
        FROM `<company data view on BigQuery>`
        WHERE 
        _table_suffix BETWEEN FORMAT_DATE('%Y%m%d',DATE_SUB(CURRENT_DATE(), INTERVAL 11 DAY))
        AND FORMAT_DATE('%Y%m%d',DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY))
        '''   
        )

        results = update_raw_query.result()     

        for row in results:
            print("{} : {} views".format(row.url, row.view_count))


    
    # "Sample code" of company "customer website behavior data" for prediction 
    # Generate ML features from "Google Analytics raw data" (total feautures generated: about 120)
    def update_pivot(self):

        update_pivot_query = self.client.query(
        f'''
        CREATE OR REPLACE TABLE 
        `<company data table on BigQuery>` AS(

        WITH

        -- Generate data range
            input_date_array AS(
            SELECT date
            FROM UNNEST(GENERATE_DATE_ARRAY('2022-10-01', '2022-12-31')) AS date  # campaign period
            ),

            date_array AS(
            SELECT date AS snapshot_date  
            FROM input_date_array
            WHERE date = DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY)
            ),


        -- Feature 1
            feat1_relabel AS(
            SELECT
                clientId,
                visitNumber,
                hits.eventInfo.eventAction AS event_page,
                hits.eventInfo.eventCategory AS event_name,
                date AS event_date,
                CASE
                WHEN hits.eventInfo.eventLabel = "<customer behavior name>" THEN 1
                WHEN hits.eventInfo.eventLabel = "<customer behavior name>" THEN 2
                WHEN hits.eventInfo.eventLabel = "<customer behavior name>" THEN 3
                WHEN hits.eventInfo.eventLabel = "<customer behavior name>" THEN 4
                ELSE
                0
            END
                AS feat1_score
            FROM
                `<company data table on BigQuery: self.table>`,
                UNNEST (hits) AS hits
            WHERE
                hits.eventInfo.eventCategory="<customer behavior category>"),

            feat1_score AS(
            SELECT
                clientId,
                visitNumber,
                event_date,
                event_page,
                event_name,
                MAX(feat1_score) AS max_feat1_score
            FROM
                feat1_relabel
            GROUP BY 1,2,3,4,5
            ORDER BY 1,2,3,4
            ),

            feat1_aggregated_score AS(
            SELECT
                clientId,
                visitNumber,
                event_date,
                event_name,
                SUM(max_feat1_score) as feat1_score
            FROM
            feat1_score 
            GROUP BY 1,2,3,4
            ORDER BY 1,2,3,4
            ),


        -- Feature 2
            feat2_relabel AS(
            SELECT
                clientId,
                visitNumber,
                hits.eventInfo.eventCategory as event_name,
                date AS event_date,
                CASE
                WHEN hits.eventInfo.eventLabel = "<customer behavior name>" THEN 1
                WHEN hits.eventInfo.eventLabel = "<customer behavior name>" THEN 2
                ELSE
                0
            END
                AS feat2_score
            FROM
            `<company data table on BigQuery: self.table>`,
            UNNEST (hits) AS hits 
            WHERE
            hits.eventInfo.eventCategory="<customer behavior category>"
            AND hits.eventInfo.eventAction != "<customer behavior name>"),

            feat2_aggregated_score AS(
            SELECT
                clientId,
                visitNumber,
                event_date,
                event_name,
                SUM(feat2_score) AS sum_feat2_score
            FROM
            feat2_relabel
            GROUP BY 1,2,3,4
            ORDER BY 1,2,3,4
            ),

            
        -- Feature 3
            feat3_relabel AS(
            SELECT
                clientId,
                visitNumber,
                hits.eventInfo.eventAction as event_link,
                hits.eventInfo.eventCategory as event_name,
                date AS event_date,
                CASE
                WHEN hits.eventInfo.eventLabel = "<customer behavior name>" THEN 1
                WHEN hits.eventInfo.eventLabel = "<customer behavior name>" THEN 2
                WHEN hits.eventInfo.eventLabel = "<customer behavior name>" THEN 3
                WHEN hits.eventInfo.eventLabel = "<customer behavior name>" THEN 4
                WHEN hits.eventInfo.eventLabel = "<customer behavior name>" THEN 5
                ELSE
                0
            END
                AS feat3_score
            FROM
                `<company data table on BigQuery: self.table>`,
                UNNEST (hits) AS hits
            WHERE
                hits.eventInfo.eventCategory="<customer behavior category>" ),

            feat3_score AS(
            SELECT
                clientId,
                visitNumber,
                event_date,
                event_link,
                event_name,
                MAX(feat3_score) AS max_feat3_score
            FROM
                feat3_relabel
            GROUP BY 1,2,3,4,5
            ORDER BY 1,2,3,4
            ),

            feat3_aggregated_score AS(
            SELECT
                clientId,
                visitNumber,
                event_date,
                event_name,
                SUM(max_feat3_score) as feat3_score
            FROM
            feat3_score 
            GROUP BY 1,2,3,4
            ORDER BY 1,2,3,4
            ),


        -- Other features: different "button clicks" on company website (sample code)

            -- Count total clicks of buttons: only remain important buttons
            count_clicks AS(
            SELECT
                hits.eventInfo.eventCategory AS event_category,
                hits.eventInfo.eventLabel AS event_label,
                COUNT(hits.eventInfo.eventCategory) AS click_count
            FROM 
                `<company data table on BigQuery: self.table>`,
                UNNEST(hits) AS hits
            WHERE hits.eventInfo.eventCategory LIKE '%<customer behavior category>%'
            GROUP BY 1,2
            ORDER BY 3 DESC
            ),

            -- Filter
            button_names AS(
            SELECT 
                clientId,
                visitNumber,
                date AS event_date,
                hits.eventInfo.eventCategory AS event_category,
                hits.eventInfo.eventLabel AS event_label,
                CONCAT(hits.eventInfo.eventCategory, "-", hits.eventInfo.eventLabel) AS button_name,  
            FROM 
                `<company data table on BigQuery: self.table>`,
                UNNEST (hits) AS hits
            WHERE 
                hits.eventInfo.eventCategory LIKE '%<customer behavior category>%'
            ),

            -- Join
            buttons_joined AS(
            SELECT 
                button_names.clientId, 
                button_names.visitNumber, 
                button_names.event_date, 
                button_names.button_name, 
                count_clicks.click_count
            FROM button_names
            INNER JOIN count_clicks ON button_names.event_category = count_clicks.event_category
            AND button_names.event_label = count_clicks.event_label
            ),

            buttons_relabel AS(
            SELECT 
                clientId, 
                visitNumber,
                event_date,
                CASE         
                    WHEN button_name IN ('<important company website buttons to keep their names>') THEN button_name
                    ELSE 'buttons_other'  -- Other buttons
                END 
                    AS event_name,
                CASE
                    WHEN event_name IN (<the most important company website buttons>) THEN 3
                    WHEN event_name IN (<important company website buttons>) THEN 2
                    ELSE 1 
                END 
                    AS buttons_score  -- Other buttons
            FROM buttons_joined
            WHERE
                click_count >= 200  -- At least 200 clicks
            ),

            buttons_aggregated_score AS(
            SELECT 
                clientId,
                visitNumber, 
                event_date,
                event_name,
                SUM(buttons_score) AS buttons_score 
            FROM buttons_relabel
            GROUP BY 1,2,3,4
            ORDER BY 1,2,3,4
            ),


        -- Skip rest of the feature creation code, which are similar to the code above 


        -- y (purchase)
            label_y AS(
            SELECT
                clientId,
                visitNumber,
                date AS event_date,
                "label_transaction" as event_name,
                CASE
                WHEN totals.transactions is NULL THEN 0
                ELSE
                    1
                END
                    AS transaction
            FROM
                `<company data table on BigQuery: self.table>`
                ORDER BY 1,2,3
            ),


        -- Union all features created
            feature AS(
                SELECT * FROM feat1_aggregated_score
                UNION ALL
                SELECT * FROM feat2_aggregated_score
                UNION ALL
                SELECT * FROM feat3_aggregated_score
                UNION ALL
                SELECT * FROM buttons_aggregated_score 
                -- Other features skipped
            ),


        -- Cross Join with date
            feature_cross_join AS(
            SELECT
                clientId,
                visitNumber,
                event_date,
                snapshot_date,
                REGEXP_REPLACE(replace(replace(replace(replace(replace(replace(replace(event_name," ","_"),"-","_"),"/","_"),"ó","o"),"í","i"),"é","e"),"á","a"), r"[^a-zA-Z0-9]+", "_") as event_name,
                SUM(IF(DATE_DIFF(CAST(CONCAT(SUBSTRING(event_date,1,4),"-",SUBSTRING(event_date,5,2),"-",SUBSTRING(event_date,7,2)) as date), snapshot_date,Day) Between -10 and 0, session_scroll_score,0)) AS time_windowed_metrics
            FROM
                feature
            CROSS JOIN
                date_array
            GROUP BY 
                1,2,3,4,5
            ),

            label_cross_join AS(
            SELECT
                clientId,
                visitNumber,
                event_date,
                snapshot_date,
                REGEXP_REPLACE(replace(replace(replace(replace(replace(replace(replace(event_name," ","_"),"-","_"),"/","_"),"ó","o"),"í","i"),"é","e"),"á","a"), r"[^a-zA-Z0-9]+", "_") as event_name,
                SUM(IF(DATE_DIFF(CAST(CONCAT(SUBSTRING(event_date,1,4),"-",SUBSTRING(event_date,5,2),"-",SUBSTRING(event_date,7,2)) as date),snapshot_date,Day) Between 0 and 3, transaction,0)) AS time_windowed_metrics
            FROM
                label_y
            CROSS JOIN
                date_array
            GROUP BY 
                1,2,3,4,5
            ),

            all_cross_join AS(
                SELECT * FROM feature_cross_join
                UNION ALL
                SELECT * FROM label_cross_join
            ),


        -- Create pivot table (feature)
            all_cross_join_pivot AS (

                SELECT * FROM all_cross_join
                
                    PIVOT (
                        SUM((time_windowed_metrics))
                        -- Need support from python
                        FOR event_name IN ('<all features generated above>')
                )
            ),


        -- Static features
            pre_static_feat AS(
            SELECT
                clientId,
                visitNumber,
                trafficSource.keyword AS search_keyword,
                device.browser AS browser,
                device.isMobile AS is_mobile,
                device.mobileDeviceBranding AS mobile_brand,
                device.deviceCategory AS device_category,
                geoNetwork.country AS country,
                geoNetwork.region AS region,
                IFNULL(totals.pageviews,0) as PageView,
                IFNULL(totals.timeOnSite,0) as TimeOnSite,
                CASE 
                WHEN CAST(hits.eCommerceAction.action_type AS int) = 2 THEN 1
                ELSE 0 END AS view_product,  
                CASE 
                WHEN CAST(hits.eCommerceAction.action_type AS int) = 3 THEN 1
                ELSE 0 END AS add2cart, 
                CASE 
                WHEN CAST(hits.eCommerceAction.action_type AS int) = 5 THEN 1
                ELSE 0 END AS checkout,  
            FROM `<company data table on BigQuery: self.table>`,
            UNNEST(hits) AS hits
            ),

            static_feat AS(
            SELECT 
                clientId,
                visitNumber,
                search_keyword, 
                browser,
                is_mobile,
                mobile_brand,
                device_category,
                country,
                region,
                PageView,
                timeOnSite,
                SUM(view_product_or_not) AS view_amount,
                SUM(add2cart_or_not) AS cart_amount,
                SUM(checkout_or_not) AS checkout_amount,
            FROM pre_static_feat
            GROUP BY 1,2,3,4,5,6,7,8,9,10,11
            ),


        -- Merge all features
            merged_table AS(
            SELECT
                all_cross_join_pivot.*, 
                static_feat.*
            FROM 
                all_cross_join_pivot 
                JOIN static_feat ON all_cross_join_pivot.clientId = static_feat.clientId 
                AND all_cross_join_pivot.visitNumber = static_feat.visitNumber
            )

        SELECT * from merged_table
        ORDER BY clientId, visitNumber, event_date, snapshot_date
        )
        '''
        )
        
        results = update_pivot_query.result()     

        for row in results:
            print("{} : {} views".format(row.url, row.view_count))

### Import BigQuery data to Python

In [ ]:
class Queries:

    def __init__(self, static_dataset, behavior_dataset):
        self.static_dataset = static_dataset
        self.behavior_dataset = behavior_dataset
        self.client = bq.Client()

    # static query
    def static_query(self):
        static_query = f'''
        WITH
            pre_features AS(
            SELECT 
                clientId,
                PARSE_DATE('%Y%m%d', date) AS event_date,
                visitNumber AS visit_number,
                trafficSource.medium AS medium,
                CASE WHEN CAST(hits.eCommerceAction.action_type AS int) = 6 THEN 1
                ELSE 0 END AS purchase_or_not,  
            FROM 
                `<company data table on BigQuery: self.table>`,
                UNNEST(hits) AS hits
            ),

            x_features AS(
            SELECT 
                clientId,
                event_date,
                visit_number,
                medium,
                SUM(purchase_or_not) AS purchase_amount,
            FROM pre_features
            GROUP BY 1,2,3,4
            -- HAVING COUNT(clientId) > 1
            )

        SELECT * FROM x_features
        ORDER BY clientId, event_date, visit_number ASC
        '''

        static_table = self.client.query(static_query).result().to_arrow(create_bqstorage_client=True).to_pandas()

        return (static_table, static_table.shape)


    # behavior query
    # This queried table is the feature data updated with function "update_pivot" above
    # For both "history" and "realtime" data (updated daily)
    def behavior_query(self):
        behavior_query = f'''
        SELECT * FROM `<company data table on BigQuery: self.table>` 
        ORDER BY clientId, visitNumber, event_date, snapshot_date
        '''
        
        behavior_table = self.client.query(behavior_query).result().to_arrow(create_bqstorage_client=True).to_pandas()

        return (behavior_table, behavior_table.shape)

### Return analysis
Another customer website behavior created with Python

In [ ]:
# Return Analysis: customer behavior for their last 2 website visits

class Return_analysis:
    
    def __init__(self, static_df, type):
        self.static_df = static_df
        self.type = type
    
    # Data pre-processing
    def pre_processing(self):
        self.static_df['event_date'] = pd.to_datetime(self.static_df['event_date'], format='%Y-%m-%d')

        visit_num = self.static_df.value_counts('clientId').reset_index()
        print('# of total clientIds:', visit_num.shape[0])  # unique client num
        only_once = visit_num[visit_num[0] == 1]['clientId'].to_list()
        
        for_preprocess = self.static_df.sort_values(['clientId', 'visit_number'], ascending=False).reset_index(drop=True)
        for_preprocess.fillna(0, inplace=True)  

        return (for_preprocess, only_once)

    
    # Return analysis for "purchasers"
    def RA_purchasers(self, for_preprocess):

        # Purchasers with at least 2 visits: can do return analysis
        clients_purchased = for_preprocess[for_preprocess['purchase_amount'] > 0].value_counts('clientId').reset_index()['clientId'].to_list()
        purchases = for_preprocess[for_preprocess['clientId'].isin(clients_purchased)].reset_index(drop=True)
        print('# of data of clients who purchase:', purchases.shape[0])
        print('# of total purchasers:', purchases.value_counts('clientId').shape[0])
        
        for_groupby = {'clientId': 'last', 'event_date': 'last', 'visit_number': 'last', 'medium': 'last', 'purchase_amount': 'last'}

        # Check who can do return analysis and who can't
        check_first_purchase = purchases.groupby('clientId', as_index=False).agg(for_groupby)
        no_RA_id = check_first_purchase[check_first_purchase['purchase_amount'] > 0]['clientId'].to_list()  

        # Drop clients who can't do return analysis
        to_drop = purchases[purchases['clientId'].isin(no_RA_id)]  
        purchases_RA = purchases[~purchases['clientId'].isin(no_RA_id)]
        to_drop = to_drop.groupby('clientId', as_index=False).apply(lambda x: x.iloc[:-1]).reset_index(drop=True)  
        purchases_RA.reset_index(drop=True)
        purchases_RA = pd.concat([purchases_RA, to_drop]).reset_index(drop=True)  

        # Select purchasers
        latest_purchases = purchases_RA[(purchases_RA['purchase_amount'] > 0)]  
        latest_purchases_idx = list(latest_purchases.index)
        before_latest_purchases_idx = [idx + 1 for idx in latest_purchases_idx]
        final_purchase_idx = sorted(latest_purchases_idx + before_latest_purchases_idx, reverse=False)
        return_purchases = purchases_RA.iloc[final_purchase_idx, :]  

        # Exclude exceptions (minority)
        outliers = return_purchases.value_counts('clientId').reset_index()[return_purchases.value_counts('clientId').reset_index()[0] == 1]['clientId'].to_list()
        return_purchases = return_purchases[~return_purchases['clientId'].isin(outliers)]
        print('# of purchasers who does RA:', return_purchases.value_counts('clientId').shape[0])

        # Calculate time period between 2 visits & last medium
        return_purchases.reset_index(drop=True, inplace=True)
        purchase_date = return_purchases[return_purchases.index%2 == 0][['clientId', 'event_date', 'visit_number','medium']]
        pre_purchase_date = return_purchases[return_purchases.index%2 != 0][['clientId', 'event_date', 'visit_number', 'medium']]
        purchase_date.reset_index(drop=True, inplace=True), pre_purchase_date.reset_index(drop=True, inplace=True)
        
        x_RA = pd.concat([purchase_date, pre_purchase_date], axis=1)
        x_RA.columns = ['clientId', 'latest event date', 'latest visit num', 'lastest medium', 'clientId1', 'second latest event date', 'visit_number1', 'second latest medium']
        x_RA.drop(['clientId1', 'visit_number1'], axis=1, inplace=True)
        x_RA['days since last visit'] = (x_RA['latest event date'] - x_RA['second latest event date']).dt.days
        
        # Purchasers who can't do return analysis: purchased at first visit
        x_RA_clients = x_RA.value_counts('clientId').reset_index()['clientId'].to_list()
        latest_purchases1 = purchases[~purchases['clientId'].isin(x_RA_clients)]
        latest_purchases1 = latest_purchases1.drop_duplicates('clientId', keep='last')  # only remain purchaser data
        print('# of purchasers who cannot do RA: ', latest_purchases1.shape[0])

        final_purchase_idx1 = list(latest_purchases1.index)
        new_purchases = purchases.iloc[final_purchase_idx1, :]
        x_RA1 = new_purchases[['clientId', 'event_date', 'visit_number','medium']]
        x_RA1.columns = ['clientId', 'latest event date', 'latest visit num', 'lastest medium']

        x_RA.reset_index(drop=True, inplace=True), x_RA1.reset_index(drop=True, inplace=True)
        return_analysis = pd.concat([x_RA, x_RA1])

        return return_analysis


    # Return analysis for "non-purchasers" 
    def RA_non_purchasers(self, for_preprocess, only_once, RA_purchasers):

        clients_purchased = for_preprocess[for_preprocess['purchase_amount'] > 0].value_counts('clientId').reset_index()['clientId'].to_list() 
        no_purchase = for_preprocess[~for_preprocess['clientId'].isin(clients_purchased)].reset_index(drop=True) 
        print('# of total data of all non-purchasers:', no_purchase.shape)

        # non-purchasers who can do return analysis
        latest_no_purchase = no_purchase[~no_purchase['clientId'].isin(only_once)]
        print('# of data of non-purchasers who visit more than once:', latest_no_purchase.shape[0])

        latest = latest_no_purchase.drop_duplicates('clientId', keep='first')
        latest_index = list(latest.index)
        latest.drop('purchase_amount', axis=1, inplace=True)
        latest.columns = ['clientId', 'latest event date', 'latest visit num', 'lastest medium']

        for_second_latest = latest_no_purchase.drop(latest_index)
        second_latest = for_second_latest.drop_duplicates('clientId', keep='first')
        second_latest.drop(['visit_number', 'purchase_amount'], axis=1, inplace=True)
        second_latest.columns = ['clientId', 'second latest event date', 'second latest medium']

        x_RA = latest.merge(second_latest, left_on='clientId', right_on='clientId', how='inner', right_index=False)
        x_RA['days since last visit'] = (x_RA['latest event date'] - x_RA['second latest event date']).dt.days

        # non-purchasers who can't do return analysis
        latest_no_purchase1 = no_purchase[no_purchase['clientId'].isin(only_once)]
        print('# of data of non-purchasers who only visit once:', latest_no_purchase1.shape[0])

        x_RA1 = latest_no_purchase1[['clientId', 'event_date', 'visit_number','medium']]
        x_RA1.columns = ['clientId', 'latest event date', 'latest visit num', 'lastest medium']

        x_RA.reset_index(drop=True, inplace=True), x_RA1.reset_index(drop=True, inplace=True)
        return_analysis = pd.concat([x_RA, x_RA1])


        # history data: do sampling to control # of data
        if self.type == 'history':
            percentage = (RA_purchasers.value_counts('clientId').reset_index(drop=True).shape[0]/0.01)/return_analysis.shape[0]
            print('training data sampling percentage: ' + str(round(percentage*100, 2)) + ' %')
            new_return_analysis = return_analysis.sample(frac=percentage, random_state=0, axis=0).reset_index(drop=True)

            print('# of total non-purchasers before sampling:', return_analysis.shape[0])
            print('# of total non-purchasers after sampling: ', new_return_analysis.shape[0])
            return new_return_analysis

        else:
            print('# of total non-purchasers: ', return_analysis.shape[0])
            return return_analysis

    
    # Union purchaser and non-purchaser
    def merge(self, RA_purchasers, RA_non_purchasers):
        RA_purchasers.rename(columns={'visit_number_x': 'visit_number'}, inplace=True)
        final_x_static_table = pd.concat([RA_non_purchasers, RA_purchasers])
        final_x_static_table = final_x_static_table.sample(frac=1, random_state=0, axis=0).reset_index(drop=True) 

        final_x_static_table['medium combination'] = final_x_static_table['lastest medium'] + ' , ' + final_x_static_table['second latest medium']

        # time delta --> days since last visit
        final_x_static_table = final_x_static_table.rename({'time delta': 'days since last visit'})

        final_x_static_table.sort_values('clientId', ascending=False, inplace=True)
        
        return final_x_static_table

### Data cleansing

In [ ]:
class Data_cleansing:
    
    def __init__(self, static_df, behavior_df, type):
        self.static_df = static_df
        self.behavior_df = behavior_df
        self.type = type


    # Merge static data & behavior data + cleaning
    def merge(self):
        self.behavior_df['event_date'] = pd.to_datetime(self.behavior_df['event_date'], format='%Y%m%d')
        x_pre_table = self.behavior_df.merge(self.static_df, left_on=['clientId', 'visitNumber'], right_on=['clientId', 'latest visit num'], how='inner')
        print(x_pre_table.shape)

        # Drop useless columns
        x_pre_table.drop(['latest event date', 'second latest event date', 'time_delta', 'latest visit num'], axis=1, inplace=True)
        print('# of data before dropping TimeOnSite < AVG rows:', x_pre_table.shape[0])

        x_pre_table = x_pre_table[x_pre_table['Pageview'] > 1]
        print('# of data after dropping Pageview <= 1 rows:', x_pre_table.shape[0])

        # Drop 0 columns
        print('# of columns before dropping all NaN columns:', x_pre_table.shape[1])
        x_pre_table = x_pre_table.dropna(axis=1, how='all')
        print('# of columns after dropping all NaN columns:', x_pre_table.shape[1])

        # Address NaN
        values = {'second latest medium': 'unknown', 'days since last visit': 180}
        x_pre_table.fillna(value=values, inplace=True)

        naming_dict = {'lastest medium': 'medium',  
                    'second latest medium': 'medium1',}
        x_pre_table.rename(columns=naming_dict, inplace=True)

        x_pre_table.isna().sum()[x_pre_table.isna().sum() > 0].sort_values(ascending=False).to_frame(name='Num of NA') 

        # Data distribution
        print(x_pre_table.shape)
        x_pre_table.value_counts('label_transaction').head()

        # Fill NaNs
        x_pre_table.fillna(0, inplace=True)
        x_pre_table.isna().sum()[x_pre_table.isna().sum() > 0].sort_values(ascending=False).to_frame(name='Num of NA')  # 沒有 NaN 了

        # realtime data: Drop people who already purchased
        if self.type == 'realtime':
            purchasers = x_pre_table[x_pre_table['label_transaction'] > 0]['clientId'].tolist()
            x_pre_table = x_pre_table[~x_pre_table['clientId'].isin(purchasers)]
            print('# of purchasers dropped: ', len(purchasers))

        return x_pre_table  # final table

### Continue data processing

In [ ]:
class Data_pre_processing:
    
    def __init__(self, x_pre_table, dummy_cols, type):

        self.x_pre_table = x_pre_table[x_pre_table['days since last visit'] >= 0]
        
        self.x_pre_table['region'] = self.x_pre_table['region'].map({'Taiwan': 'region_Taiwan'}, na_action='ignore')
        self.x_pre_table['country'] = self.x_pre_table['country'].map({'Taiwan': 'country__Taiwan'}, na_action='ignore')
        
        self.dummy_cols = dummy_cols
        self.type = type

    
    # Get dummy variable & Drop useless columns
    def get_dummy(self):
        
        # prepare data
        clientId = self.x_pre_table['clientId'].reset_index(drop=True)
        data = self.x_pre_table.drop(['clientId', 'snapshot_date', 'event_date', 'medium1', 'visitNumber'], axis=1)  

        for column in self.dummy_cols:
            dummy_df = pd.get_dummies(data[column])
            for col in dummy_df.columns:
                if dummy_df[col].sum() < self.x_pre_table.shape[0] * 0.01:  # Remove features with very few counts
                    dummy_df.drop(col, axis=1, inplace=True)
            data.reset_index(drop=True, inplace=True), dummy_df.reset_index(drop=True, inplace=True)
            data = pd.concat([data, dummy_df], axis=1)
            data.drop(column, axis=1, inplace=True)
        
        data.drop(0, axis=1, inplace=True)  
        data.drop('(not set)', axis=1, inplace=True)
        y_data = data['label_transaction'].to_frame()
        x_data = data.drop('label_transaction', axis=1)

        if self.type == 'history':
            return (x_data, y_data)  # Don't need to map client back to training (history) data
        else:
            return (x_data, y_data, clientId)  # Map clientId back to real-time data for advertising


    # Cut data
    def cut_data(self, X_data, y_data):
        if self.type == 'history':
            X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)
            X_train, X_stack, y_train, y_stack = train_test_split(X_train, y_train, test_size=0.4, random_state=42)
            print(X_train.shape)
            print(y_train.shape)
            print(X_test.shape)
            print(y_test.shape)

            return (X_train, y_train, X_test, y_test, X_stack, y_stack)
        else:
            print(X_data.shape)
            print(y_data.shape)
            
            return(X_data, y_data) 

### Model evaluation
* Select the best model to predict

In [ ]:
# Different metrics

class Model_evaluation:

    def __init__(self, y_test, y_predict, y_proba):
        self.y_test = y_test
        self.y_predict = y_predict
        self.y_proba = y_proba

    # Balanced accuracy score
    def balanced_acc(self):
        accuracy = balanced_accuracy_score(self.y_test, self.y_predict)

        return accuracy


    # AUC area
    def AUC_area(self):
        y_true = np.array(self.y_test)
        y_score = np.array(self.y_proba)
        roc_auc = roc_auc_score(y_true, y_score[:, 1])

        return roc_auc


    # Confusion matrix
    def cf_matrix(self):
        cf_matrix = confusion_matrix(self.y_test, self.y_predict)
        # print(cf_matrix)
        
        ax = sns.heatmap(cf_matrix, annot=True, cmap='Blues')
        ax.set_title('Confusion Matrix with labels\n\n')
        ax.set_xlabel('\nPredicted Values')
        ax.set_ylabel('Actual Values ')

        ## Ticket labels - List must be in alphabetical order
        ax.xaxis.set_ticklabels(['False','True'])
        ax.yaxis.set_ticklabels(['False','True'])

        ## Display the visualization of the Confusion Matrix.
        plt.show()

### Models
* Model selection: history (training) data

In [ ]:
# Different models

class Models:

    def __init__(self, X_train, y_train, X_test, y_test, X_stack, y_stack, type):
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        self.X_stack = X_stack
        self.y_stack = y_stack
        self.type = type

    
    # gradient boosting
    def gb(self):
        boosting = GradientBoostingClassifier(max_depth=5).fit(self.X_train, self.y_train)
        boosting_prob = boosting.predict_proba(self.X_test)
        boosting_pred = boosting.predict(self.X_test)

        if self.type == 'history':
            return(boosting, boosting_prob, boosting_pred)
        else:
            return(boosting, boosting_pred)

    
    # random forest
    def rf(self):
        forest = RandomForestClassifier(max_depth=5).fit(self.X_train, self.y_train)
        forest_prob = forest.predict_proba(self.X_test)
        forest_pred = forest.predict(self.X_test)

        if self.type == 'history':
            return(forest, forest_prob, forest_pred)
        else:
            return(forest, forest_pred)



    # XGBoost
    def xgb(self):
        xgboost = XGBClassifier(max_depth=5).fit(self.X_train, self.y_train)
        xgboost_prob = xgboost.predict_proba(self.X_test)
        xgboost_pred = xgboost.predict(self.X_test)

        if self.type == 'history':
            return(xgboost, xgboost_prob, xgboost_pred)
        else:
            return(xgboost, xgboost_pred)

    
    # Stacking
    def stacking(self, boosting, forest, xgboost):
        level_0 = [('Gradient Boosting', boosting), ('Random Forest', forest), ('XGBoost', xgboost)]   

        level_1 = StackingClassifier(estimators=level_0, final_estimator=LogisticRegression())
        stack = level_1.fit(self.X_stack, self.y_stack)

        stack_prob = stack.predict_proba(self.X_test)
        stack_pred = stack.predict(self.X_test)

        if self.type == 'history':
            return(stack_prob, stack_pred)
        else:
            return stack_pred

### Hyper-parameter tuning 

In [ ]:
class Tune_models:

    def __init__(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train

    
    def gb(self):
        gb = GradientBoostingClassifier(random_state=42)
        gb_param_grid = {'learning_rate': np.logspace(-3, 0, 10, endpoint=True),
                        'n_estimators': [100, 300, 500],
                        'max_depth': [1, 2, 3]}

        search_gb = GridSearchCV(estimator=gb,
                            param_grid=gb_param_grid,
                            cv=5,
                            scoring='balanced_accuracy',
                            n_jobs=-1,
                            verbose=1)

        search_gb.fit(self.X_train, self.y_train)

        return(search_gb.best_score_, search_gb.best_estimator_)


    def rf(self):
        rf = RandomForestClassifier(random_state=42)
        rf_param_grid = {'n_estimators': [100, 300, 500],
                        'criterion': ['gini', 'entropy'],
                        'max_depth': [10, 20, 30],
                        'max_features': ['sqrt', 0.6, 0.7, 0.8, None]}

        search_rf = GridSearchCV(estimator=rf,
                            param_grid=rf_param_grid,
                            cv=5,
                            scoring='balanced_accuracy',
                            n_jobs=-1,
                            verbose=1)

        search_rf.fit(self.X_train, self.y_train)

        return(search_rf.best_score_, search_rf.best_estimator_)


    def xgb(self):
        xgb = XGBClassifier()
        xgb_param_grid = {'learning_rate': np.logspace(-3, 0, 10, endpoint=True),
                        'max_depth': [3, 5, 8],  # 不能太高
                        'min_child_weight': [1, 3, 5],  # 不能太低
                        'gamma': [0, 0.05, 0.1],  
                        'colsample_bytree': [0.5, 0.7, 0.9]}

        search_xgb = GridSearchCV(estimator=xgb,
                            param_grid=xgb_param_grid,
                            cv=5,
                            scoring='balanced_accuracy',
                            n_jobs=-1,
                            verbose=1)

        search_xgb.fit(self.X_train, self.y_train)

        return (search_xgb.best_score_, search_xgb.best_estimator_)

### Output & Relabel

In [ ]:
class Export_output:

    def __init__(self, model_prob, clientId, file_name, snapshot_date):
        self.model_prob = model_prob
        self.clientId = clientId
        self.file_name = file_name
        self.snapshot_date = snapshot_date


    # Create an empty csv file to write in the re-scored outputs to upload
    def create_csv(self):

        df = pd.DataFrame(list())
        df.to_csv(self.file_name + '.csv')
        
        return self.file_name + '.csv'  # For API file location


    # Create an empty excel file to write in the original scores of outputs
    def create_excel(self):
        workbook = Workbook()
        workbook.save(self.file_name + '.xlsx')


    def output(self, n):  # n: the criteria for "next purchasers"
        propensity = [i[1] for i in self.model_prob.tolist()]
        propensity_df = pd.DataFrame(propensity, columns=['purchase propensity'])
        clientId_df = self.clientId.to_frame().reset_index(drop=True)
        output = pd.concat([clientId_df, propensity_df], axis=1)
        output = output.sort_values('purchase propensity', ascending=False)
        output.reset_index(inplace=True, drop=True)

       # Next purchasers
        n_num = output.shape[0] * n
        n_score = output.loc[int(n_num), 'purchase propensity']
        print('# of high propensity customers:', n_num)
        output['relabeled propensity'] = output['purchase propensity'].apply(lambda x: 0.16 if x >= n_score else 0)
        output.to_excel(self.file_name + '.xlsx')

        output_new = output[['clientId', 'relabeled propensity']]
        output_new = output_new[output_new['relabeled propensity'] > 0]
        output_new['relabeled propensity_'] = output_new['relabeled propensity'].astype(str)
        
        output_new['relabeled propensity_'] = output_new['relabeled propensity_'] + self.snapshot_date
        output_new.drop('relabeled propensity', axis=1, inplace=True)
        output_new['relabeled propensity_'] = output_new['relabeled propensity_'].astype(float)
        
        print('# of high propensity leads:', output_new.shape)
        output_new.to_csv(self.file_name + '.csv', index=False)
        
        return output_new

### Training / history data

#### Data preparation + Data pre-processing

In [ ]:
# Update realtime data before import any data
realtime_raw_data = '<company data table on BigQuery>'

update_data = Update_data(realtime_raw_data)
# update_data.update_raw()
update_data.update_pivot()

In [ ]:
# Import data from GBQ
t_static_data = '<company data table on BigQuery>'
t_behavior_data = '<company data table on BigQuery>'

t_queries = Queries(t_static_data, t_behavior_data)
t_static, t_static_shape = t_queries.static_query()
t_behavior, t_behavior_shape = t_queries.behavior_query()

In [ ]:
t_RA = Return_analysis(t_static, 'history')
t_static, t_only_once = t_RA.pre_processing()
print('# of clientIds only visit once:', len(t_only_once)) 
print('# of total static data:', t_static.shape)

t_RA_purchasers = t_RA.RA_purchasers(t_static)  # purchasers
print('# of total purchasers: ', t_RA_purchasers.value_counts('clientId').shape[0])

t_RA_non_purchasers = t_RA.RA_non_purchasers(t_static, t_only_once, t_RA_purchasers)  # non-purchasers

# merge purchasers & non-purchasers
t_final_static_table = t_RA.merge(t_RA_purchasers, t_RA_non_purchasers)
print(t_final_static_table.shape)
print('# of total RA clients:', t_final_static_table.value_counts('clientId').shape[0])

In [ ]:
# Data cleansing
t_data_cleansing = Data_cleansing(t_final_static_table, t_behavior, 'history')
t_pre_table = t_data_cleansing.merge()  # final table before data pre-processing
t_pre_table.isna().sum()[t_pre_table.isna().sum() > 0].sort_values(ascending=False).to_frame(name='Num of NA')  


# Data pre-processing
dummy_cols = ['medium', 'medium combination', 'search_keyword', 'browser', 'is_mobile', 'mobile_brand', 'device_category', 'country', 'region']
t_data_preprocess = Data_pre_processing(t_pre_table, dummy_cols, 'history')
t_x_data, t_y_data = t_data_preprocess.get_dummy()
X_train, y_train, X_test, y_test, X_stack, y_stack =  t_data_preprocess.cut_data(t_x_data, t_y_data)

#### Train models + Model selection

In [ ]:
# Train diffferent models
train_models = Models(X_train, y_train, X_test, y_test, X_stack, y_stack, 'history')
boosting, gb_y_prob, gb_y_pred = train_models.gb()
forest, rf_y_prob, rf_y_pred = train_models.rf()
xgboost, xgb_y_prob, xgb_y_pred = train_models.xgb()
stack_y_prob, stack_y_pred = train_models.stacking(boosting, forest, xgboost)

Model evaluation: select gradient boosting as an example

In [ ]:
# gradient boosting
gb_evaluation = Model_evaluation(y_test, gb_y_pred, gb_y_prob)
gb_roc = gb_evaluation.AUC_area()
print('Gradient boosting AUC area: ', gb_roc) 
print('Gradient boosting balanced accuracy: ', gb_evaluation.balanced_acc())  # Gradient boosting balanced accuracy: 0.8339378395448561
gb_evaluation.cf_matrix()

In [ ]:
# random forest
rf_evaluation = Model_evaluation(y_test, rf_y_pred, rf_y_prob)
rf_roc = rf_evaluation.AUC_area()
print('Random forest AUC area: ', rf_roc)
print('Random forest balanced accuracy score: ', rf_evaluation.balanced_acc())  # Random forest balanced accuracy score:  0.6649750991427331
rf_evaluation.cf_matrix()

In [ ]:
# XGBoost
xgb_evaluation = Model_evaluation(y_test, xgb_y_pred, xgb_y_prob)
xgb_roc = xgb_evaluation.AUC_area()
print('XGBoost AUC area: ', xgb_roc)
print('XGBoost balanced accuracy score: ', xgb_evaluation.balanced_acc())  # XGBoost balanced accuracy score:  0.836694660577354
xgb_evaluation.cf_matrix()

In [ ]:
# Stacking
stack_evaluation = Model_evaluation(y_test, stack_y_pred, stack_y_prob)
stack_roc = stack_evaluation.AUC_area()
print('Stacking AUC area: ', stack_roc)
print('Stacking accuracy score: ', stack_evaluation.balanced_acc())  # Stacking accuracy score:  0.8357770498754344
stack_evaluation.cf_matrix()

Select Gradient Boosting in this campaign

### Realtime data: X_real

In [ ]:
# Address feature difference
def discard_diff(X_train, X_test):
    diff_cols = list(set(X_train.columns).symmetric_difference(set(X_test.columns)))
    X_train.drop(diff_cols, axis=1, inplace=True, errors='ignore')
    X_test.drop(diff_cols, axis=1, inplace=True, errors='ignore')

    return X_train, X_test

In [ ]:
# Export the prediction of the best model
def output(model_prob, clientId):
    propensity = [i[1] for i in model_prob.tolist()]
    propensity_df = pd.DataFrame(propensity, columns=['purchase propensity'])

    clientId_df = clientId.to_frame().reset_index(drop=True)
    output = pd.concat([clientId_df, propensity_df], axis=1)
    
    return output

#### Data preparation + Data pre-processing

In [ ]:
# Import data from GBQ
r_static_data = '<company data table on BigQuery>'
r_behavior_data = '<company data table on BigQuery>'

r_queries = Queries(r_static_data, r_behavior_data)
r_static, r_static_shape = r_queries.static_query()
r_behavior, r_behavior_shape = r_queries.behavior_query()

# Check if the data is updated
print('behavior data max date: ', r_behavior['event_date'].max())
print('behavior data min date: ', r_behavior['event_date'].min())
print('behavior data snapshot date: ', r_behavior['snapshot_date'].min())

In [ ]:
# Return analysis
r_RA = Return_analysis(r_static, 'realtime')
r_static, r_only_once = r_RA.pre_processing()
print('# of clientIds only visit once:', len(r_only_once)) 
print('# of total static data:', r_static.shape)

r_RA_purchasers = r_RA.RA_purchasers(r_static)  # purchasers
print('# of total purchasers: ', r_RA_purchasers.value_counts('clientId').shape[0])

r_RA_non_purchasers = r_RA.RA_non_purchasers(r_static, r_only_once, r_RA_purchasers)  # non-purchasers

# merge purchasers & non-purchasers
r_final_static_table = r_RA.merge(r_RA_purchasers, r_RA_non_purchasers)
print(r_final_static_table.shape)
print('# of total RA clients:', r_final_static_table.value_counts('clientId').shape[0])

In [ ]:
# Data cleansing
r_data_cleansing = Data_cleansing(r_final_static_table, r_behavior, 'realtime')
r_pre_table = r_data_cleansing.merge()  # final table before data pre-processing
r_pre_table.isna().sum()[r_pre_table.isna().sum() > 0].sort_values(ascending=False).to_frame(name='Num of NA')  

# Data pre-processing
r_data_preprocess = Data_pre_processing(r_pre_table, dummy_cols, 'realtime')
r_x_data, r_y_data, clientId = r_data_preprocess.get_dummy()
X_real, y_real =  r_data_preprocess.cut_data(r_x_data, r_y_data)

In [ ]:
# Merge all the datasets as final training data
X_train_new = pd.concat([X_train, X_test, X_stack])
y_train_new = pd.concat([y_train, y_test, y_stack])
X_train_new, X_real = discard_diff(X_train_new, X_real)
print(X_train_new.shape)
print(y_train_new.shape)
print(X_real.shape)

#### Predict with the selected model + Export output

In [ ]:
# Choose gradient boosting in this campaign
boosting_real = GradientBoostingClassifier(max_depth=5).fit(X_train_new, y_train_new)
boosting_prob_real = boosting_real.predict_proba(X_real)
boosting_pred_real = boosting_real.predict(X_real)

In [ ]:
# Gradient Boosting feature importance
features = list(X_train_new.columns)
boosting_real_feat = boosting_real.feature_importances_
boosting_real_feat_df = pd.DataFrame({'features': features}).join(pd.DataFrame({'weights': boosting_real_feat}))
gb_feat_imp_real = boosting_real_feat_df.sort_values('weights', ascending=False)
gb_feat_imp_real.head(10)

#### Output & Output Relabel

In [ ]:
today = date.today()
yesterday = today - timedelta(days = 1)
snapshot_date = str(yesterday.strftime('%m%d'))
file_name = 'client_propensity_' + snapshot_date

In [ ]:
# update automatically every day
today = date.today()
yesterday = today - timedelta(days = 1)
snpshot_date = str(yesterday.strftime('%m%d'))
print('snapshot date: ', snapshot_date)

output_gb = Export_output(boosting_prob_real, clientId, file_name, snapshot_date)
csv_location_name = output_gb.create_csv()  
output_gb.create_excel()
output_new = output_gb.output()

output_new.head()  # Predicion result: Upload and Check with "Management api"